In [110]:
from utils.dataloader import dtLoader
import numpy as np
import pywt
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize

In [111]:
source = "/Users/pradithaalwis/Projects/University of Peradeniya/Fetal Mov Data/Ward/"
drop_zero_path = "/Users/pradithaalwis/Projects/University of Peradeniya/Fetal Mov Data//Randomly_Dropped_Windows/"
sensors = ['ax1', 'ay1', 'az1', 'ax2', 'ay2', 'az2', 'ax3', 'ay3', 'az3', 'ax4', 'ay4', 'az4']
sample_freq = 32

# Stride in seconds
stride = 1

# Window length in seconds
window_length = 8

# Safety margin in seconds
front_margin = 2
rear_margin = 2

In [112]:
"""
Splits the signals into windows
Args:
    SOURCE          : path to the sensor data
    SAMPLE_FREQ     : sample frequency of sensors
    STRIDE          : stride used for windowing (in seconds)
    WINDOW_LENGTH   : length of the required window in seconds
    FRONT_MARGIN    : Required offset before a fetal kick (in seconds)
    REAR_MARGIN     : Required offset after a fetal kick (in seconds)
"""
dataloader = dtLoader(SOURCE=source,
                        SAMPLE_FREQ=sample_freq,
                        STRIDE=stride,
                        WINDOW_LENGTH=window_length,
                        FRONT_MARGIN=front_margin,
                        REAR_MARGIN=rear_margin,
                        SENSORS=sensors)

In [113]:
"""
Args:
Returns:
    self.window_list            : Set of generated windows
    self.kick_count_list        : Number of kicks in each window
    counts                      : Dictionary containing number of windows                             with the given number of kicks
"""
counts = dataloader.split_windows()

--------------------------------------------------------------------------------
These are the list of files in the source directory.
['W17_R1.csv', 'W09_R1.csv', 'W05_R1.csv', 'W21_R1.csv', 'W05_R2.csv', 'W21_R2.csv', 'W15_R2.csv', 'W19_R1.csv', 'W07_R1.csv', 'W15_R1.csv', 'W23_R1.csv', 'W11_R3.csv', 'W27_R2.csv', 'W11_R2.csv', 'W27_R1.csv', 'W11_R1.csv', 'W03_R1.csv', 'W25_R1.csv', 'W11_R5.csv', 'W29_R1.csv', 'W01_R1.csv', 'W11_R4.csv', 'W13_R1.csv', 'W25_R2.csv', 'W29_R2.csv', 'W29_R3.csv', 'W25_R3.csv', 'W04_R2.csv', 'W20_R1.csv', 'W08_R1.csv', 'W16_R1.csv', 'W04_R1.csv', 'W22_R1.csv', 'W30_R1.csv', 'W06_R1.csv', 'W18_R1.csv', 'W14_R1.csv', 'W22_R2.csv', 'W10_R1.csv', 'W02_R1.csv', 'W26_R1.csv', 'W24_R3.csv', 'W24_R2.csv', 'W12_R1.csv', 'W24_R1.csv', 'W28_R1.csv']
--------------------------------------------------------------------------------
Length of dataframe :  1286  seconds
W17_R1.csv has been processed
Length of dataframe :  1263  seconds
W09_R1.csv has been processed
Length

In [114]:
window_list, num_kick_list = dataloader.drop_zero_kicks(counts[0] - counts[1], drop_zero_path)

In [115]:
print(window_list.shape)

(3710, 12, 256)


In [116]:
def gen_scalo(signal):
    values = np.geomspace(1, 32, num=64)
    widths = []
    sample_freq = 32
    frequencies = pywt.scale2frequency('cmor2.5-0.5', values) * sample_freq

    for i in range(len(values)):
        if frequencies[i] > 2:
            widths.append(values[i])
    C = 0.5
    B = 2.5
    cwtmatr, freqs = pywt.cwt(window_list[0][2], widths, 'cmor' + str(B) + '+' + str(C), sampling_period= 1 / sample_freq, method='fft')
    cwtmatr = np.abs(cwtmatr[:-1, :-1]) ** 2
    return cwtmatr, freqs

In [117]:
# Should randomize this selection
scalograms = []
for i in range(len(num_kick_list)):
    temp = []
    for sensor in window_list[i]:
        cwtmatr, freq = gen_scalo(sensor)
        normed_matrix = normalize(cwtmatr, axis=1, norm='l1')
        temp.append(cwtmatr)
    temp = np.array(temp, dtype=np.float32)
    scalograms.append(temp)
scalograms = np.array(scalograms, dtype=np.float32)

In [121]:
print(min(num_kick_list))

-43


In [15]:
print(scalograms.shape)

(3640, 12, 37, 255)


In [81]:
import torch
import torchvision
from torchvision import transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [87]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

Files already downloaded and verified


In [93]:
# temp = next(iter(trainloader))

2


In [85]:
print(temp[1].shape)

torch.Size([4])


In [104]:
import torch
import torchvision
from torchvision import transforms
class CustomToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        # Swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        print(sample[1].shape, sample[1])
        return (torch.from_numpy(sample[0]).to(torch.float32).to("mps"), torch.from_numpy(sample[1]).to(torch.long))
class CustomNormalize(object):
    """Normalize an image with mean and standard deviation."""
    def __init__(self, mean, std):
        self.mean = torch.tensor(mean).to(torch.float32).to("mps")
        self.std = torch.tensor(std).to(torch.float32).to("mps")

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.

        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.sub_(m).div_(s)
        return tensor
transform = transforms.Compose(
    [CustomToTensor(),
    CustomNormalize(mean=[0.5] * 12, std=[0.5] * 12)])

In [109]:
print(scalograms.shape)
print(min(num_kick_list))
trainset = [transform((scalogram, gt)) for (scalogram, gt) in zip(scalograms, num_kick_list)]

(3640, 12, 37, 255)
-43
() 0


TypeError: expected np.ndarray (got numpy.longlong)

torch.Size([12, 37, 255])


In [75]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

RuntimeError: _share_filename_: only available on CPU

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(len(sensors), 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()